In [4]:
# The Oracle 
# V1.9.6 2024-07-17
#
import sys
import os
import json

# This line imports the print_help function from a module named common_utils, which is located within a package called utils. 
# The print_help function is likely used to display help messages to the user.
from utils.common_utils import print_help
# This line is importing several functions from the ioc_functions module, which is part of the ioc_processing package. 
# These functions are probably related to the processing of Indicators of Compromise (IOCs) in cybersecurity
from ioc_processing.ioc_functions import (
    validate_iocs,
    parse_bulk_iocs,
    #bulk_analysis,
    process_individual_ioc_file,
    analysis,
    extract_last_analysis_date,
    format_date,
    calculate_total_malicious_score
)
#from ioc_processing.chinmay_bulkanalysis import bulk_analysis
from file_operations.file_utils import (
    read_file,
    clean_input
)
from ioc_processing.final_verdict import (
    load_report,
    save_report,
    is_recent,
    analyze_ioc_reports,
    append_verdicts_to_report
)

In [ ]:
def main():
    script_directory = "/home/jovyan/Infra-Validate - Mine"
    input_directory = os.path.join(script_directory, 'input_files')
    output_directory = os.path.join(script_directory, 'output_files')
    show_help = False

    if len(sys.argv) > 1 and sys.argv[1] in ['-h', '--help']:
        show_help = True

    while True:
        if show_help:
            print_help()
            show_help = False
            continue

        choice = validate_iocs()
        if choice is None:
            continue
        if choice == '5':
            sys.exit("\nExiting script.")

        output_to_file = input("Do you want to save the output to a file? (yes/no): ").strip().lower() == 'yes'
        output_file_path = None

        if output_to_file:
            output_file_name = input('Please provide the name of the output file for IOC results: ')
            output_file_path = os.path.join(output_directory, output_file_name)

        aggregated_report = ""

        # if choice == '4':
        #     data_source = input("Enter 'file' to use a file or 'raw' to input raw IOCs: ").strip().lower()

        #     if data_source == 'file':
        #         file_name = input('Please provide the name of the txt file for Bulk IOCs: ')
        #         file_path = os.path.join(input_directory, file_name)
        #         content = read_file(file_path)
        #     elif data_source == 'raw':
        #         content = input('Please input the Bulk IOCs, separated by new lines: ')
        #     else:
        #         print("Invalid input. Please choose 'file' or 'raw'.")
        #         continue

            cleaned_content = clean_input(content)
            iocs = parse_bulk_iocs(cleaned_content)
            # bulk_combined_reports = bulk_analysis(iocs, output_file_path)

            # Output for bulk analysis
            # print("\n--- All Scans Completed ---\n")
            # aggregated_report = bulk_combined_reports["plaintext_output"]

        elif choice in ['1', '2', '3']:
            ioc_type = {'1': 'ips', '2': 'urls', '3': 'hashes'}[choice]
            data_source = input(f"Enter 'file' to use a file or 'raw' to input raw {ioc_type.upper()}: ").strip().lower()

            if data_source == 'file':
                file_name = input(f'Please provide the name of the txt file for {ioc_type.upper()}: ')
                file_path = os.path.join(input_directory, file_name)
                iocs = process_individual_ioc_file(file_path, ioc_type)
            elif data_source == 'raw':
                content = input(f'Please input the {ioc_type.upper()}, separated by new lines: ')
                cleaned_content = clean_input(content)
                iocs = {ioc_type: cleaned_content.splitlines()}
            else:
                print("Invalid input. Please choose 'file' or 'raw'.")
                continue

            aggregated_report = analysis(iocs, output_file_path)

        if output_to_file:
            with open(output_file_path, "a") as outfile:
                outfile.write(aggregated_report)
                outfile.write("\n")
        else:
            print(aggregated_report)

# Entry point of the script.
if __name__ == "__main__":
    main()  # Call the main function directly